<a href="https://colab.research.google.com/github/kittisaktaoma/colab_practice/blob/master/Test_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Library 



In [ ]:
pip install keras

In [ ]:
pip install keras-gcn

In [ ]:
pip install numpy

In [ ]:
pip install matplotlib

In [ ]:
pip install scipy

In [ ]:
pip install scipy